In [222]:
# нью рок нью джаз
import numpy as np
import pandas as pd

In [223]:
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

imputer = SimpleImputer(strategy="most_frequent")

In [224]:
path_train = "train"

wag_prob = pd.read_parquet(path_train + "/wagons_probeg_ownersip.parquet").convert_dtypes()
target = pd.read_csv(path_train +'/target/y_train.csv').convert_dtypes()
to_predict = pd.read_csv(path_train +'/target/y_predict.csv').convert_dtypes()

# добавляем kri_izm
kri_izm = pd.read_parquet(path_train+'/kti_izm.parquet').convert_dtypes()

In [225]:
target.head()

,wagnum,month,target_month,target_day
0,33361,2023-01-01,0,0
1,33364,2023-01-01,0,0
2,33366,2023-01-01,0,0
3,33358,2023-01-01,0,0
4,33349,2023-01-01,0,0


### Предобработка основной таблицы с вагонами

In [226]:
train_target = target[target["month"] == "2023-01-01"].drop("month", axis=1)
train = wag_prob[wag_prob["repdate"] < "2023-01-01"]

features = train.groupby("wagnum").agg(
    {
        "ost_prob": ["mean", "last", "max", 'var'],
        "manage_type": "last",
        "rod_id": "last",
        "reestr_state": "last",
    }
)
features.columns = ["_".join(col).strip() for col in features.columns.values]
features = features.reset_index()


### Предобработка параметров вагонов

In [227]:

wag_param = pd.read_parquet(path_train + "/wag_params.parquet").convert_dtypes()

m_train = wag_param.drop(
    ["model", "tipvozd", "date_build", "srok_sl", "date_iskl"], axis=1
)

### Предобработка КТИ

In [228]:
kti_izm = pd.read_parquet('train/kti_izm.parquet').convert_dtypes()
kti_izm.head()

,wagnum,operation_date_dttm,mileage_all,axl1_l_w_flange,axl1_r_w_flange,axl2_l_w_flange,axl2_r_w_flange,axl3_l_w_flange,axl3_r_w_flange,axl4_l_w_flange,axl4_r_w_flange,axl1_l_w_rim,axl1_r_w_rim,axl2_l_w_rim,axl2_r_w_rim,axl3_l_w_rim,axl3_r_w_rim,axl4_l_w_rim,axl4_r_w_rim
0,325,2022-08-01,112091,30.5,31.0,26.7,27.4,28.8,28.6,26.3,27.4,67.5,65.6,66.7,64.5,61.5,60.3,67.9,66.8
1,325,2022-08-03,112471,30.4,31.4,27.5,27.6,28.9,29.0,25.5,27.8,65.7,65.8,65.7,63.7,61.6,61.4,67.4,69.3
2,325,2022-08-05,113938,30.0,31.1,27.1,27.9,28.5,28.9,26.2,27.6,70.8,69.5,65.1,65.6,62.2,60.7,65.7,69.6
3,325,2022-09-15,121071,30.5,31.4,26.4,27.3,28.9,28.4,24.9,28.0,70.2,68.1,67.4,65.8,62.3,63.0,70.9,70.3
4,325,2022-09-09,117341,30.7,31.5,26.9,27.1,28.9,28.5,25.5,27.5,68.1,64.1,64.5,64.9,61.6,61.1,68.7,<NA>


In [229]:
# from matplotlib import pyplot as plt
# 
# selected_wagnum = kti_izm['wagnum'].unique()[0]
# selected_df = kti_izm[kti_izm['wagnum'] == selected_wagnum]
# 
# # Получение списка параметров, исключая 'wagnum' и 'operation_date'
# parameters = selected_df.columns.drop(['wagnum', 'operation_date_dttm'])
# 
# # Определение размера сетки графиков
# n_params = len(parameters)
# n_cols = 3  # Количество колонок графиков
# n_rows = (n_params + n_cols - 1) // n_cols  # Количество строк графиков
# 
# fig, axs = plt.subplots(n_rows, n_cols, figsize=(15, n_rows * 5))  # Размер фигуры в дюймах
# 
# # Создание графиков в цикле
# for i, param in enumerate(parameters):
#     row = i // n_cols
#     col = i % n_cols
#     ax = axs[row, col] if n_rows > 1 else axs[col]
#     ax.plot(selected_df['operation_date_dttm'], selected_df[param], label=param)
#     ax.set_title(f'Изменение {param} со временем')
#     ax.set_xlabel('Дата операции')
#     ax.set_ylabel(param)
#     ax.grid(True)
#     ax.legend()
# 
# # Удаление пустых мест для графиков, если параметров меньше, чем мест в сетке
# for i in range(n_params, n_rows * n_cols):
#     row = i // n_cols
#     col = i % n_cols
#     fig.delaxes(axs[row, col] if n_rows > 1 else axs[col])
# 
# plt.tight_layout()
# plt.show()

In [230]:
kti_izm_agg = kti_izm.drop('operation_date_dttm', axis=1).groupby("wagnum").agg(['max', 'min', 'mean', 'std', 'var', 'last'])
kti_izm_agg.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in kti_izm_agg.columns.values]
kti_izm_agg.head()

,mileage_all_max,mileage_all_min,mileage_all_mean,mileage_all_std,mileage_all_var,mileage_all_last,axl1_l_w_flange_max,axl1_l_w_flange_min,axl1_l_w_flange_mean,axl1_l_w_flange_std,...,axl4_l_w_rim_mean,axl4_l_w_rim_std,axl4_l_w_rim_var,axl4_l_w_rim_last,axl4_r_w_rim_max,axl4_r_w_rim_min,axl4_r_w_rim_mean,axl4_r_w_rim_std,axl4_r_w_rim_var,axl4_r_w_rim_last
wagnum,,,,,,,,,,,,,,,,,,,,,
0,106028,91119,98028.555556,4875.873514,23774142.527778,106028,27.2,26.3,26.788889,0.236878,...,39.855556,2.424413,5.877778,37.1,47.3,38.1,41.388889,3.724730,13.873611,39.2
20,32522,22506,26816.5,4271.509140,18245790.333333,32522,30.5,29.1,30.1,0.668331,...,73.975,0.287228,0.0825,74.0,74.9,73.3,74.1,0.673300,0.453333,73.9
23,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,30.6,29.7,30.1,0.352767,...,36.24,1.030857,1.062667,35.5,38.3,35.1,36.6,1.019804,1.04,35.7
36,105942,102329,105038.75,1806.500000,3263442.25,105942,24.2,23.8,23.975,0.170783,...,33.325,0.556028,0.309167,33.1,34.8,33.7,34.325,0.464579,0.215833,33.7
40,35797,35797,35797.0,0.000000,0.0,35797,26.9,26.3,26.6,0.424264,...,28.3,0.989949,0.98,27.6,29.6,26.5,28.05,2.192031,4.805,29.6


In [231]:


merged = features.merge(m_train, on="wagnum", how="inner")
merged = train_target.merge(merged, on="wagnum", how="left")

In [232]:
merged.head()

,wagnum,target_month,target_day,ost_prob_mean,ost_prob_last,ost_prob_max,ost_prob_var,manage_type_last,rod_id_last,reestr_state_last,...,tara,zavod_build,cnsi_probeg_dr,cnsi_probeg_kr,kuzov,telega,tormoz,tippogl,norma_km,ownertype
0,33361,0,0,40308.352941,153692,159999,4462370209.914088,0,1,1,...,245,6,110,160,2,2,2,11,160000,0
1,33364,0,0,36773.124183,157438,159999,1860819655.491056,0,1,1,...,245,6,110,160,2,2,2,11,160000,0
2,33366,0,0,30174.464052,159999,159999,3164737251.579291,0,1,1,...,245,6,110,160,2,2,2,11,160000,0
3,33358,0,0,34319.823529,28597,41827,17934407.712074,0,1,1,...,245,6,110,160,2,2,2,11,160000,0
4,33349,0,0,85561.934641,150043,159999,5815367655.324646,0,1,1,...,245,6,110,160,2,2,2,11,160000,0


In [233]:

# добавление КТИ
# merged = pd.merge(merged, kti_izm_agg, on='wagnum', how='left')
# #merged.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in merged.columns.values]
# 
# merged['is_kti_found'] = merged['mileage_all_max'].notnull().astype(int)



In [234]:
merged.head()

,wagnum,target_month,target_day,ost_prob_mean,ost_prob_last,ost_prob_max,ost_prob_var,manage_type_last,rod_id_last,reestr_state_last,...,tara,zavod_build,cnsi_probeg_dr,cnsi_probeg_kr,kuzov,telega,tormoz,tippogl,norma_km,ownertype
0,33361,0,0,40308.352941,153692,159999,4462370209.914088,0,1,1,...,245,6,110,160,2,2,2,11,160000,0
1,33364,0,0,36773.124183,157438,159999,1860819655.491056,0,1,1,...,245,6,110,160,2,2,2,11,160000,0
2,33366,0,0,30174.464052,159999,159999,3164737251.579291,0,1,1,...,245,6,110,160,2,2,2,11,160000,0
3,33358,0,0,34319.823529,28597,41827,17934407.712074,0,1,1,...,245,6,110,160,2,2,2,11,160000,0
4,33349,0,0,85561.934641,150043,159999,5815367655.324646,0,1,1,...,245,6,110,160,2,2,2,11,160000,0


In [235]:

column_names = merged.columns

merged = imputer.fit_transform(merged)

merged = pd.DataFrame(merged, columns=column_names)

X_train = merged.drop(["wagnum"], axis=1)
y_month_name = "target_month"
y_day_name = "target_day"
target_names = [y_month_name, y_day_name]

In [236]:
print("Training for month and day prediction")

logreg_month = LogisticRegression()
logreg_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
logreg_day = LogisticRegression()
logreg_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])

rf_month = RandomForestClassifier()
rf_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
rf_day = RandomForestClassifier()
rf_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])

cb_month = CatBoostClassifier()
cb_month.fit(X_train.drop(target_names, axis=1), X_train[y_month_name])
cb_day = CatBoostClassifier()
cb_day.fit(X_train.drop(target_names, axis=1), X_train[y_day_name])

Training for month and day prediction
Learning rate set to 0.046422
0:	learn: 0.5801360	total: 10.3ms	remaining: 10.3s
1:	learn: 0.5121765	total: 20.5ms	remaining: 10.2s
2:	learn: 0.4444986	total: 31.5ms	remaining: 10.5s
3:	learn: 0.3802808	total: 42.7ms	remaining: 10.6s
4:	learn: 0.3391415	total: 51.9ms	remaining: 10.3s
5:	learn: 0.3065893	total: 62.3ms	remaining: 10.3s
6:	learn: 0.2752930	total: 72.2ms	remaining: 10.2s
7:	learn: 0.2509417	total: 82.5ms	remaining: 10.2s
8:	learn: 0.2227078	total: 92.3ms	remaining: 10.2s
9:	learn: 0.2070054	total: 100ms	remaining: 9.91s
10:	learn: 0.1912791	total: 110ms	remaining: 9.93s
11:	learn: 0.1749909	total: 120ms	remaining: 9.85s
12:	learn: 0.1621644	total: 128ms	remaining: 9.74s
13:	learn: 0.1525137	total: 137ms	remaining: 9.63s
14:	learn: 0.1433886	total: 145ms	remaining: 9.53s
15:	learn: 0.1362773	total: 153ms	remaining: 9.43s
16:	learn: 0.1299409	total: 162ms	remaining: 9.37s
17:	learn: 0.1268796	total: 171ms	remaining: 9.35s
18:	learn: 0.12

In [237]:
path_test = 'test'
test_target = pd.read_csv(path_test+'/target/y_test.csv').convert_dtypes().drop("month", axis=1)
#test = wag_prob[wag_prob["repdate"] >= "2023-01-01"]
test = wag_prob

In [240]:
# test_target = target[target["month"] == "2023-01-01"].drop("month", axis=1)
# test = wag_prob[wag_prob["repdate"] < "2023-01-01"]

test_features = test.groupby("wagnum").agg(
    {
        "ost_prob": ["mean", "last", "max", 'var'],
        "manage_type": "last",
        "rod_id": "last",
        "reestr_state": "last",
    }
)
test_features.columns = ["_".join(col).strip() for col in test_features.columns.values]
test_features = test_features.reset_index()

m_test = wag_param.drop(
    ["model", "tipvozd", "date_build", "srok_sl", "date_iskl"], axis=1
)

merged = test_features.merge(m_test, on="wagnum", how="inner")
merged = test_target.merge(merged, on="wagnum", how="left")

# merged = pd.merge(merged, kti_izm_agg, on='wagnum', how='left')
# #merged.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in merged.columns.values]
# 
# merged['is_kti_found'] = merged['mileage_all_max'].notnull().astype(int)


column_names = merged.columns
merged = imputer.fit_transform(merged)
merged = pd.DataFrame(merged, columns=column_names)

X_test = merged.drop(["wagnum"], axis=1)

In [241]:
y_pred_logreg_month = logreg_month.predict(X_test.drop(target_names, axis=1))
y_pred_rf_month = rf_month.predict(X_test.drop(target_names, axis=1))
y_pred_cb_month = cb_month.predict(X_test.drop(target_names, axis=1))

y_pred_logreg_day = logreg_day.predict(X_test.drop(target_names, axis=1))
y_pred_rf_day = rf_day.predict(X_test.drop(target_names, axis=1))
y_pred_cb_day = cb_day.predict(X_test.drop(target_names, axis=1))

print("Month score:")
f1_logreg_month = f1_score(X_test[y_month_name], y_pred_logreg_month)
print(f"Logistic Regression month F1 Score: {f1_logreg_month}")
f1_rf_month = f1_score(X_test[y_month_name], y_pred_rf_month)
print(f"Random Forest month F1 Score: {f1_rf_month}")
f1_cb_month = f1_score(X_test[y_month_name], y_pred_cb_month)
print(f"CatBoost month F1 Score: {f1_cb_month}")

print("Day score:")
f1_logreg_day = f1_score(X_test[y_day_name], y_pred_logreg_day)
print(f"Logistic Regression day F1 Score: {f1_logreg_day}")
f1_rf_day = f1_score(X_test[y_day_name], y_pred_rf_day)
print(f"Random Forest day F1 Score: {f1_rf_day}")
f1_cb_day = f1_score(X_test[y_day_name], y_pred_cb_day)
print(f"CatBoost day F1 Score: {f1_cb_day}")

Month score:
Logistic Regression month F1 Score: 0.1669307976756471
Random Forest month F1 Score: 0.48377793799567415
CatBoost month F1 Score: 0.49982275788727404
Day score:
Logistic Regression day F1 Score: 0.0
Random Forest day F1 Score: 0.208955223880597
CatBoost day F1 Score: 0.26378896882494


In [242]:
def correct_days_with_month(day_array, month_array):
    corrected_day = day_array.copy()
    corrected_day[month_array == 0] = 0
    return corrected_day


y_pred_logreg_day_corrected = correct_days_with_month(y_pred_logreg_day, y_pred_logreg_month)
y_pred_rf_day_corrected = correct_days_with_month(y_pred_rf_day, y_pred_rf_month)
y_pred_cb_day_corrected = correct_days_with_month(y_pred_cb_day, y_pred_cb_month)


print("Day score after applying month info to day:")
f1_logreg_day = f1_score(X_test[y_day_name], y_pred_logreg_day_corrected)
print(f"Logistic Regression day F1 Score: {f1_logreg_day}")
f1_rf_day = f1_score(X_test[y_day_name], y_pred_rf_day_corrected)
print(f"Random Forest day F1 Score: {f1_rf_day}")
f1_cb_day = f1_score(X_test[y_day_name], y_pred_cb_day_corrected)
print(f"CatBoost day F1 Score: {f1_cb_day}")


Day score after applying month info to day:
Logistic Regression day F1 Score: 0.0
Random Forest day F1 Score: 0.19087136929460582
CatBoost day F1 Score: 0.25521472392638034


In [243]:
y_pred_logreg_day

array([0., 0., 0., ..., 0., 0., 0.])

In [244]:
X_test.drop(target_names, axis=1).columns

Index(['ost_prob_mean', 'ost_prob_last', 'ost_prob_max', 'ost_prob_var',
       'manage_type_last', 'rod_id_last', 'reestr_state_last', 'rod_id',
       'gruz', 'cnsi_gruz_capacity', 'cnsi_volumek', 'tara', 'zavod_build',
       'cnsi_probeg_dr', 'cnsi_probeg_kr', 'kuzov', 'telega', 'tormoz',
       'tippogl', 'norma_km', 'ownertype'],
      dtype='object')

In [245]:
feature_importances = cb_month.get_feature_importance()
feature_names = X_test.drop(target_names, axis=1).columns
importance_dict = {feature_names[i]: importance for i, importance in enumerate(feature_importances)}
dict(sorted(importance_dict.items(), key=lambda item: item[1], reverse=True))

{'ost_prob_mean': 21.122889046006325,
 'ost_prob_var': 16.5005202484614,
 'ost_prob_last': 14.107830579837303,
 'ost_prob_max': 14.102215841269407,
 'tara': 4.91644429044897,
 'cnsi_gruz_capacity': 4.47139578588512,
 'gruz': 4.161477535072675,
 'cnsi_volumek': 4.006510475624941,
 'zavod_build': 3.788271371939553,
 'rod_id': 2.5648402942365838,
 'rod_id_last': 2.110446793489058,
 'tippogl': 1.8593286673247966,
 'tormoz': 1.3881365270964778,
 'norma_km': 1.2015856817380586,
 'kuzov': 1.1236028626658556,
 'telega': 1.0647765433696457,
 'cnsi_probeg_dr': 1.04665586258974,
 'manage_type_last': 0.2239853669602112,
 'ownertype': 0.1887733988702467,
 'cnsi_probeg_kr': 0.04119820487016637,
 'reestr_state_last': 0.009114622243470307}

In [246]:
# import os
# import h2o
# import xgboost as xgb
# import pandas as pd
# from h2o.automl import H2OAutoML
# 
# h2o.init()
# 
# model_path = ""
# model_file = os.path.join(model_path, "StackedEnsemble_AllModels_1_AutoML_2_20231111_112553")
# train = h2o.H2OFrame(X_train)
# test = h2o.H2OFrame(X_test)
# train[y_name] = train[y_name].asfactor()
# test[y_name] = test[y_name].asfactor()
# 
# x = train.columns
# y = y_name
# x.remove(y)
#     
# # if os.path.exists(model_file):
# #     print("sdfsd")
# #     aml = h2o.load_model(model_file)
# # else:
# aml = H2OAutoML(max_models=15, seed=42)
# aml.train(x=x, y=y, training_frame=train)
# 
# h2o.save_model(aml.leader, path=model_path)
# 
# lb = aml.leaderboard
# print(lb.head(rows=lb.nrows))

In [247]:
# y_pred = aml.predict(test).as_data_frame().squeeze()
# result = f1_score(X_test[y_name], y_pred["predict"])
# print(f"F1: {result}")


In [248]:
# h2o.save_model(aml.leader, path=f"saved_models/h20_{result}")
# h2o.shutdown()

In [249]:
# feature_importance = logreg.coef_
# 
# avg_feature_importance = np.mean(np.abs(feature_importance), axis=0)
# 
# feature_importance_df = pd.DataFrame(
#     {"Feature": X_train.columns, "Importance": avg_feature_importance}
# )
# 
# feature_importance_df = feature_importance_df.sort_values(
#     by="Importance", ascending=False
# )
# 
# print(feature_importance_df)

In [250]:
# feature_importance = rf.feature_importances_
# 
# feature_importance_df = pd.DataFrame(
#     {"Feature": X_train.columns, "Importance": feature_importance}
# )
# 
# feature_importance_df = feature_importance_df.sort_values(
#     by="Importance", ascending=False
# )
# 
# print(feature_importance_df)

In [ ]:
# import shap
# 
# explainer = shap.TreeExplainer(rf)
# shap_values = explainer.shap_values(X_train)
# shap.summary_plot(shap_values, X_train, plot_type="bar")